# Assignment_Week3: Segmenting and Clustering neighborhoods in Toronto

In [3]:
!pip install BeautifulSoup4
!pip install requests

Step 1

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [15]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')


table = soup.find("table")
table_rows = table.tbody.find_all("tr")

In [16]:
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    data.append(row)

In [17]:
df = pd.DataFrame(data, columns = ["Postal Code", "Borough", "Neighborhood"])
df = df[~df['Postal Code'].isnull()]
df

,Postal Code,Borough,Neighborhood
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
176,M5Z\n,Not assigned\n,Not assigned\n
177,M6Z\n,Not assigned\n,Not assigned\n
178,M7Z\n,Not assigned\n,Not assigned\n
179,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [21]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

df.drop(df[df['Borough']=="Not assigned\n"].index,axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
1,M1B\n,Scarborough\n,"Malvern, Rouge\n"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
4,M1G\n,Scarborough\n,Woburn\n
5,M1H\n,Scarborough\n,Cedarbrae\n


In [38]:
df["Postal Code"] = df["Postal Code"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")

In [39]:
# group more than one neighborhood in one postal code area, seperate with a comma
df = df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()

In [33]:
df.Neighborhood.str.count("Not assigned").sum()

0

In [25]:
print("Shape: ", df.shape)

Shape:  (103, 3)


In [111]:
pip install geocoder

     |████████████████████████████████| 98 kB 6.7 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [27]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0



body = client_bdbfafe8c47b4772b0aef3fb2a2056cd.get_object(Bucket='capstoneprojectcourse-donotdelete-pr-lxhgnnueu8ziwq',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
df_combined = df.join(df_data_1.set_index('Postal Code'), on='Postal Code', how='inner')
df_combined

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
